# DrainageAI Demo - Google Colab Integration with BYOL (Unlabeled Data Focus)

This notebook demonstrates the DrainageAI workflow using Google Colab's GPU acceleration, focusing on the BYOL approach with unlabeled data only. It includes support for grayscale (single-channel) images.

## Step 1: Check GPU Availability

In [ ]:
import torch

print(f"GPU available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
else:
    print("WARNING: No GPU detected. Processing will be slow.")

## Step 2: Install Dependencies

In [ ]:
!pip install rasterio geopandas scikit-image matplotlib pytorch-lightning torch-geometric

## Step 3: Clone the DrainageAI Repository

In [ ]:
# Clone the repository
!git clone https://github.com/yourusername/DrainageAI.git

%cd DrainageAI

## Step 4: Upload Test Imagery

In [ ]:
from google.colab import files

print("Please upload your multispectral imagery file (GeoTIFF format):")
uploaded = files.upload()

# Get the filename of the uploaded file
imagery_filename = list(uploaded.keys())[0]
print(f"Uploaded file: {imagery_filename}")

## Step 5: Create Output Directory

In [ ]:
!mkdir -p colab_results

## Step 6: Check Image Channels

In [ ]:
import rasterio

# Check the number of channels in the uploaded image
with rasterio.open(imagery_filename) as src:
    num_channels = src.count
    print(f"Image has {num_channels} channel(s)")
    
# Determine which model to use based on the number of channels
if num_channels < 3:
    print("This is a grayscale or 2-channel image. We'll use the grayscale-compatible model.")
    recommended_model = "grayscale-byol"
else:
    print("This is a multi-channel image. We'll use the standard model.")
    recommended_model = "byol"

## Step 7: Calculate Spectral Indices

In [ ]:
print("\n=== Step 1: Calculate Spectral Indices ===\n")

!python main.py indices --imagery {imagery_filename} --output colab_results/indices.tif --indices ndvi,ndmi,msavi2

## Step 8: Run Drainage Detection with Grayscale Support

In [ ]:
print("\n=== Step 2: Detect Drainage Pipes ===\n")

# Choose one of the following model options based on the image type:

# For grayscale images (1 channel) or 2-channel images
if num_channels < 3:
    print("Using grayscale-compatible BYOL model...")
    !python main.py detect --imagery {imagery_filename} --indices colab_results/indices.tif --output colab_results/drainage_grayscale_byol.tif --model grayscale-byol
# For RGB images (3+ channels)
else:
    print("Using standard BYOL model...")
    !python main.py detect --imagery {imagery_filename} --indices colab_results/indices.tif --output colab_results/drainage_byol.tif --model byol

## Step 9: Vectorize Results

In [ ]:
print("\n=== Step 3: Vectorize Results ===\n")

# Determine which detection result to use based on the model used
if num_channels < 3:
    detection_file = "colab_results/drainage_grayscale_byol.tif"
else:
    detection_file = "colab_results/drainage_byol.tif"

!python main.py vectorize --input {detection_file} --output colab_results/drainage_lines.shp

## BYOL Workflow for Unlabeled Data

This section demonstrates the BYOL (Bootstrap Your Own Latent) approach using only unlabeled data for pretraining. It supports both RGB and grayscale images.

### Upload Unlabeled Data

In [ ]:
# Create directories for unlabeled data
!mkdir -p data/unlabeled/imagery

# Upload unlabeled imagery (you can upload multiple files)
print("Please upload unlabeled imagery files (GeoTIFF format):")
uploaded_unlabeled = files.upload()

# Save uploaded files to the unlabeled directory
for filename in uploaded_unlabeled.keys():
    with open(f"data/unlabeled/imagery/{filename}", 'wb') as f:
        f.write(uploaded_unlabeled[filename])
    print(f"Saved {filename} to data/unlabeled/imagery/")

### Check Image Channels and Determine Approach

In [ ]:
import rasterio
import numpy as np
import os

# Check all images in the unlabeled directory
has_grayscale = False
for filename in os.listdir('data/unlabeled/imagery'):
    if filename.endswith(('.tif', '.tiff')):
        input_path = os.path.join('data/unlabeled/imagery', filename)
        
        # Check number of channels
        with rasterio.open(input_path) as src:
            num_channels = src.count
            print(f"Image {filename} has {num_channels} channel(s)")
            
            if num_channels < 3:
                has_grayscale = True

# Determine which approach to use
if has_grayscale:
    print("\nDetected grayscale images. We have two options:")
    print("1. Use the grayscale-compatible BYOL model directly (recommended)")
    print("2. Convert grayscale images to RGB format")
    approach = input("Which approach do you want to use? (1/2): ")
else:
    print("\nAll images have 3 or more channels. Using standard BYOL model.")
    approach = "0"  # Not using grayscale approach

### Option 1: Use Grayscale-Compatible BYOL Model

In [ ]:
if approach == "1" or not has_grayscale:
    print("\n=== Using Grayscale-Compatible BYOL Model ===\n")
    
    # Run the grayscale BYOL example
    !python examples/grayscale_byol_example.py \
        --optical-dir data/unlabeled/imagery \
        --output-dir colab_results \
        --epochs 20 \
        --test-image {imagery_filename}

### Option 2: Convert Grayscale to RGB

In [ ]:
if approach == "2":
    print("\n=== Converting Grayscale Images to RGB ===\n")
    
    # Create directory for RGB images
    !mkdir -p data/unlabeled_rgb
    
    # Function to convert grayscale to RGB
    def convert_grayscale_to_rgb(input_path, output_path):
        with rasterio.open(input_path) as src:
            # Read the data
            data = src.read()
            profile = src.profile.copy()
            
            # Check if it's already multi-channel
            if data.shape[0] >= 3:
                print(f"Image {input_path} already has {data.shape[0]} channels, skipping.")
                return False
            
            # Create 3-channel image by duplicating the grayscale channel
            if data.shape[0] == 1:
                rgb_data = np.repeat(data, 3, axis=0)
            else:
                # If it has 2 channels, add a third one
                zeros = np.zeros_like(data[0:1])
                rgb_data = np.concatenate([data, zeros], axis=0)
            
            # Update profile for RGB output
            profile.update(count=3)
            
            # Write the RGB image
            with rasterio.open(output_path, 'w', **profile) as dst:
                dst.write(rgb_data)
            
            return True
    
    # Convert all grayscale images
    for filename in os.listdir('data/unlabeled/imagery'):
        if filename.endswith(('.tif', '.tiff')):
            input_path = os.path.join('data/unlabeled/imagery', filename)
            output_path = os.path.join('data/unlabeled_rgb', filename)
            
            # Check number of channels
            with rasterio.open(input_path) as src:
                num_channels = src.count
                
                if num_channels < 3:
                    print(f"Converting {filename} to RGB format...")
                    convert_grayscale_to_rgb(input_path, output_path)
                else:
                    print(f"Copying {filename} (already has {num_channels} channels)...")
                    # Copy the file as is
                    with open(input_path, 'rb') as src_file, open(output_path, 'wb') as dst_file:
                        dst_file.write(src_file.read())
    
    print("\n=== BYOL Pretraining with Converted RGB Images ===\n")
    
    # Run BYOL pretraining with converted RGB images
    !python examples/byol_mvp_workflow.py \
        --optical-dir data/unlabeled_rgb \
        --output-dir colab_results \
        --byol-epochs 20

### BYOL Inference with Pretrained Model

In [ ]:
print("\n=== BYOL Inference with Pretrained Model ===\n")

# Run inference with the appropriate model
if approach == "1" or not has_grayscale:
    # Using grayscale-compatible model
    !python main.py detect \
        --imagery {imagery_filename} \
        --output colab_results/detection_result.tif \
        --model grayscale-byol \
        --weights colab_results/grayscale_byol_model.pth
elif approach == "2":
    # Using standard model with converted RGB images
    !python main.py detect \
        --imagery {imagery_filename} \
        --output colab_results/detection_result.tif \
        --model byol \
        --weights colab_results/byol_pretrained.pth